In [9]:
import warnings
warnings.filterwarnings("ignore")
from qdrant_client import QdrantClient
import ollama
from config import *
from common import *

In [10]:
# Initialize Ollama client
oclient = ollama.Client(host=Config.HOSTNAME)
# Initialize Qdrant client
qclient = QdrantClient(host=Config.HOSTNAME, port=Config.QDRANT_PORT)

In [11]:
import json
import requests
from typing import Optional
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM

class OllamaLLM(LLM):
    """
    A custom LLM integration for Ollama's API.
    """

    model: str = "tinyllama"  # Default model
    base_url: str = "http://localhost:11434"  # Default Ollama endpoint

    def _call(self, prompt: str, stop: Optional[list] = None) -> str:
        """
        Handles the interaction with the Ollama API.
        """
        payload = {"model": self.model, "prompt": prompt}
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",
                json=payload,
                stream=True,  # Enable streaming
            )
            response.raise_for_status()

            # Process streamed responses
            full_response = ""
            for line in response.iter_lines(decode_unicode=True):
                if line:  # Skip empty lines
                    try:
                        data = json.loads(line)
                        #print("Streaming JSON Object:", data)  # Debugging
                        #print(data)
                        full_response += data.get("response", "")
                        if data.get("done", False):  # Stop when done
                            break
                    except json.JSONDecodeError as e:
                        print(f"Failed to decode line: {line}. Error: {e}")
                        continue

            return full_response
        except requests.RequestException as e:
            raise ValueError(f"Error communicating with Ollama API: {e}")
        except ValueError as e:
            raise ValueError(f"Error processing the response: {e}")

    @property
    def _identifying_params(self) -> dict:
        """Returns identifying parameters for serialization."""
        return {"model": self.model, "base_url": self.base_url}

    @property
    def _llm_type(self) -> str:
        """Type of the LLM."""
        return "ollama"

# Instantiate the Ollama LLM
ollama_llm = OllamaLLM(model="tinyllama", base_url="http://localhost:11434")

In [13]:
ask_query = " Can you tell me something about AI Agents in healthcare"
model = "tinyllama"
query_embed , _ = get_embeddings(model, [ask_query], llm_client=oclient)
query_embed = query_embed[0]

hits = retrieve_context(query_embed=query_embed, vdb_client=qclient)
context = ""
for hit in hits:
    #print(hit.payload, "score:", hit.score)
    context += hit.payload['text']

#print(context)


# Create a PromptTemplate
topic = "AI agents in healthcare"
#query = quert_text

prompt = """
You are an assistant who answers questions based on provided context and a specific topic.
Keep the response succinct under 50 words. Exclude the context and query in final answer 
context and query text in the answer.

Context:
{context}

Topic:
{topic}

Question:
{query}

Answer:
"""

prompt_template = PromptTemplate(
    input_variables=["context", "topic", "query"],  # Variables to inject
    template=prompt,
)

# chain prompt and llm
chain = prompt_template | ollama_llm
# Run the chain with a specific input
result = chain.invoke({'context': context, 'topic': topic, 'query': ask_query})
print("LangChain Response:", result)

>>>>>>> localhost 11434
>>>>>>> localhost 6333
LangChain Response: Yes, absolutely! AI Agents in Healthcare are transforming the way care is delivered, enabling real-time analysis and action to improve patient outcomes. These agents use machine learning and natural language processing to identify patterns, analyze data, and make informed decisions based on physiological information and medical history. They provide personalized guidance and assistance for patients, reducing hospital readmissions, improving access to care, and increasing satisfaction with healthcare services. AI Agents in Healthcare have the potential to transform the way people access and receive healthcare by improving clinical decision-making, predicting patient outcomes, and providing proactive recommendations tailored to individual patient needs. By implementing AI agents in healthcare, organizations can reduce waiting times for appointments, improve communication between healthcare providers and patients, and enha

In [21]:

new_query = "Who is the author of the paper Utility of Accounting for Human Beliefs about AI Intention in Human-AI Collaboration"
topic = "Accounting for Human Beliefs"

new_query += topic


new_embed, _ = get_embeddings(model, [new_query], llm_client=oclient)
new_embed = new_embed[0]

hits = retrieve_context(new_embed, vdb_client=qclient)
new_context = ""
for hit in hits:
    new_context += hit.payload['text']

print(new_context)

result = chain.invoke({'context': new_context, 'topic': topic, 'query': new_query})
print(result)

>>>>>>> localhost 11434
>>>>>>> localhost 6333
uted quantum
applications, over Internet-scale distances.

arXiv:2208.00733v1  [cs.ET]  1 Aug 2022
IEEE IOT MAGAZINE, VOL. XX, NO. X, X 2022
1
The Rise of Quantum Internet Computing
Seng W. Loke, Member, IEEE
Abstract—This article highlights quantum Internet computing as referring to distributed quantum computing over the quantum Internet,
analogous to (classical) Internet computing involving (classical) distributed computing over the (classical) Internet. Relevant to
quantum Internet computing mation, and can be
thought of as a two-state, or two-levelled, quantum-mechanical system,
such as an electron’s spin, where the two levels are spin up and spin
down, or a photon’s polarization, where the two states are the vertical
polarization and the horizontal polarization.
2. Multiple qubits at different sites can share an entangled state, a
superpositon of “specially correlated” states, to be used in distributed
algorithms.
computing over nodes